# Invocar o modelo do Bedrock para geração de texto com o prompt Zero-shot

> *Este caderno deve funcionar bem com o kernel **`Data Science 3.0`** no SageMaker Studio*

## Introdução

Neste caderno, mostramos a você como usar um LLM para gerar uma resposta de e-mail a um cliente que forneceu feedback negativo sobre a qualidade do atendimento ao cliente que recebeu do engenheiro de suporte. 

Usaremos o modelo grande do Amazon Titan Text do Bedrock com a API do Boto3. 

O prompt usado neste exemplo é chamado de prompt zero-shot porque não fornecemos nenhum exemplo de texto junto com sua classificação além do prompt.

**Observação:** *este caderno pode ser executado dentro ou fora do ambiente AWS.*

#### Contexto
Para demonstrar a capacidade de geração de texto do Amazon Bedrock, exploraremos o uso do cliente Boto3 para se comunicar com a API do Amazon Bedrock. Demonstraremos as diferentes configurações disponíveis e como um comando simples pode gerar os resultados desejados.

#### Padrão
Simplesmente ofereceremos à API do Amazon Bedrock um comando que consiste em uma tarefa, uma instrução e um comando para que o modelo interno gere um resultado sem fornecer exemplos adicionais. Nosso objetivo é demonstrar como poderosos LLMs entendem facilmente a tarefa dada e geram resultados interessantes.

![](./images/bedrock.jpg)

#### Caso de uso
Para demonstrar a capacidade de geração de modelos no Amazon Bedrock, vamos analisar o caso de uso de geração de um e-mail.

#### Persona
Você é Bob, um gerente de atendimento ao cliente na AnyCompany e alguns de seus clientes não estão satisfeitos com o atendimento ao cliente e estão fornecendo feedbacks negativos sobre o serviço prestado pelos engenheiros de atendimento ao cliente. Você gostaria de pedir suas sinceras desculpas a esses clientes pelo serviço ruim e recuperar a confiança deles. Você precisa da ajuda de um LLM para gerar um lote de e-mails amigáveis e personalizados ao sentimento do cliente da correspondência de e-mail anterior.

#### Implementação
Para atender este caso de uso, mostraremos neste caderno como gerar um e-mail de agradecimento com base no e-mail anterior do cliente. Usaremos o grande modelo Amazon Titan Text com a API do Amazon Bedrock com o cliente Boto3. 

In [ ]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## Geração de texto

Seguindo o caso de uso explicado acima, vamos preparar um comando para o serviço Amazon Bedrock gerar um e-mail. Observe que este prompt precisaria ser modificado por [formatação Humano:/Assistente: para Claude.](https://docs.anthropic.com/claude/docs/human-and-assistant-formatting)

In [ ]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""


Vamos começar usando o grande modelo Amazon Titan. O Amazon Titan grande é compatível com uma janela de contexto de cerca de 4 mil tokens e aceita os seguintes parâmetros:
- `inputText`: prompt para o LLM
- `textGenerationConfig`: estes são os parâmetros que o modelo levará em consideração ao gerar o resultado.

In [ ]:
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":4096,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

A API do Amazon Bedrock fornece a você uma API `invoke_model` que aceita o seguinte:
- `modelId`: este é o modelo de ARN de vários modelos de base disponíveis no Amazon Bedrock.
- `accept`: o tipo de solicitação de comando.
- `contentType`: o tipo de contexto do resultado.
- `body`: uma string json string que consiste no prompt e nas configurações.

Confira a [documentação](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) para obter os IDs do modelo de geração de texto disponíveis

#### Invocar o grande modelo de linguagem Amazon Titan

Primeiro, veremos como o modelo gera um resultado com base no prompt criado anteriormente.

##### Concluir a geração do resultado

In [ ]:
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [ ]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print_ww(email)


##### Transmitir a geração do resultado
Temos acima um e-mail de exemplo gerado pelo grande modelo Amazon Titan por meio da compreensão da solicitação de comando e uso de sua compreensão inerente das diferentes modalidades. Esta solicitação à API é síncrona e aguarda o modelo gerar todo o resultado.

O Bedrock também é compatível com o resultado que pode ser transmitido conforme é gerado pelo modelo em blocos. Temos abaixo um exemplo de invocação do modelo com opção de transmissão. `invoke_model_with_response_stream` retorna um `ResponseStream` que pode ser lido.

In [ ]:
output = []
try:
    
    response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

Isso ajuda a obter rapidamente o resultado do modelo e deixa o serviço concluí-lo enquanto você o lê. Isso ajuda em casos de uso em que há blocos de texto maiores que você solicita ao modelo que gere. Posteriormente, você pode combinar todos os blocos gerados para formar o resultado completo e usá-lo no seu caso de uso.

In [ ]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)

## Conclusão
Você adquiriu experiência com o uso do SDK `boto3`, que oferece uma exposição básica à API do Amazon Bedrock. Pelo uso dessa API, você viu o caso de uso de geração de um e-mail de resposta ao feedback negativo de um cliente.

### Conclusões
- Adaptar este caderno para testar diferentes modelos disponíveis através do Amazon Bedrock, como modelos Antropich Claude e AI21 Labs Jurassic.
- Alterar os prompts para seu caso de uso específico e avaliar o resultado de diferentes modelos.
- Experimentar com o comprimento do token para entender a latência e a responsividade do serviço.
- Aplicar diferentes princípios de engenharia de prompt para obter resultados melhores.

## Agradecemos sua atenção